<h1 style="text-align:center">Part 1: Getting familiar with the Robot and Environment</h1>

To start the simulation with the robot and the environment, in a web shell run 
```
roslaunch deliverybot_simulations building_world.launch
```
This will start gazebo and rviz. Open gazebo using simulation tab and rviz using graphical tools tab. Be sure to unpause the simulation before sending commands.

## 1. The Robot
<img src="images/dbot.png" alt="Robot" width="400"/>

We can control the robot link using the available services or the GUI.
```
/dbot/open_bot_door     -> Opens Bot Door
/dbot/close_bot_door    -> Closes Bot Door
/dbot/set_pusher_in     -> Pulls pusher inwards 
/dbot/set_pusher_out    -> Pushes pusher outwards
/dbot/deliver_package   -> Pushes the package out of the robot and closes door 
```

## 2. The Elevator
<img src="images/elevator.png" alt="Elevator" width="400"/>

```
/elevator/open_elevator_doors                                             -> Open Doors
/elevator/close_elevator_doors                                            -> Close Doors
/elevator/elevator_goto_floor "floor: X" # Where X is floor number [0-2]  -> Move to floor X
```


<h1 style="text-align:center">Part 2: Multimap Server</h1>

## 1. Creating multimap_server package
* Create multimap_server package
```
catkin_create_pkg multimap_server rospy message_generation message_runtime std_msgs
```
* Update maintainer tag in package.xml

## 2. Creating the MapFilePath service file
We will first create MapFilePath.srv which we will need to pass out map file path to our map server
* Create srv folder
* Create `MapFilePath.srv` file
* Add the following code to the file and our service msg is ready
```
    string map_file_path
    ---
    bool success
```
* Update CMakeLists.txt
    * Update find_package
    ```
        find_package(catkin REQUIRED COMPONENTS
          message_generation
          rospy
          std_msgs
        )
    ```
    * Find add_service_files and uncomment then put the following line.
    ```
        add_service_files(
          FILES
          MapFilePath.srv
        )
    ```
    * Uncomment the following lines - 
    ```
        generate_messages(
            DEPENDENCIES
            std_msgs
        )
    ```
    ```
        catkin_package(
            #  INCLUDE_DIRS include
            #  LIBRARIES multimap_server
             CATKIN_DEPENDS rospy std_msgs
            #  DEPENDS system_lib
        )
    ```
* `cd`  to base folder and run `catkin_make` to build our package
* run `source devel/setup.bash`
* Now we can check our service file by running `rossrv show multimap_server/MapFilePath`

## 3. Creating the multimap_server node
* Go to `multimap_server/scripts` and create `multimap_server_node.py`
* Add the following lines to the file -

```python
    #!/usr/bin/env python

    import rospy
    import subprocess
    import signal
    from multimap_server.srv import MapFilePath

    map_server = None


    def set_map(req):
        global map_server
        if map_server is not None:
            map_server.send_signal(signal.SIGINT)
        map_server = subprocess.Popen(["rosrun", "map_server", "map_server", req.map_file_path])
        return True


    if __name__ == "__main__":
        rospy.init_node("multimap_server")
        rospy.Service('/multimap_server/load_map', MapFilePath, set_map)
        rospy.loginfo("Multimap server started.")
        rospy.spin()
```
* Update CMakeLists.txt
```
    catkin_install_python(PROGRAMS
      scripts/multimap_server_node.py
      DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
    )
```
* In the scripts folder, run `chmod +x multimap_sever_node.py` to make it executable
* `cd` to base folder and run `catkin_make` to build our package
* run `source devel/setup.bash`

## 4. Testing the multimap_server

### 4.1 Setting up   MAPS_FOLDER_PATH
* Before we test the map server, we first need the maps. To get started quickly we will not create new maps now. You can find maps of the simulation in the `/home/user/catkin_ws/src/rdd21_deliverybot/deliverybot_mapping/maps`. 
* To make it easier to input the map file path, we will create an environment variable `MAPS_FOLDER_PATH` which contains the path to the folder which contains the maps.
    * `cd` to the colder contains the maps, here - `maps_folder`
    * To set the variable in one shell, we can do `export MAPS_FOLDER_PATH=$(pwd)`
    * To set permanently we can do add it to `.bashrc` file. 
        * vim ~/.bashrc
        * Add the following line to end of file `export MAPS_FOLDER_PATH=/home/user/catkin_ws/src/rdd21_deliverybot/deliverybot_mapping/maps`
    * You can check if the environment vairable has been exported correctly by first sourcing the the .bashrc file `source ~/.bashrc` and then running `echo $MAPS_FOLDER_PATH`, it should return the absolute path to your maps folder
    
### 4.2 Testing the load_map service
* In a shell, run `roscore`
* In another shell, start rviz using `rosrun rviz rviz`
* Open the graphical tools to see rviz
* Add map a display with topic `/map`

Now that we have rviz running, we can test the map_server
* In a shell run 
```
    rosrun multimap_server multimap_server_node.py
```
* In a different shell, run 
```
rosservice call /multimap_server/load_map "map_file_path: '$MAPS_FOLDER_PATH/building/lobby.yaml'"
```
This will load the lobby map which you can see in rviz. If it is not visible ensure that topic `/map` is selected in the map display. 

<img src="images/rviz_lobby.png" alt="Map of Lobby" width="400"/>

Now to load a different map say `floor` we can simply run

```
   rosservice call /multimap_server/load_map "map_file_path: '$MAPS_FOLDER_PATH/building/floor.yaml'"
```

<img src="images/rviz_floor.png" alt="Map of Floor" width="400"/>

* Now we have a server where we can easily switch between maps

---

<h1 style="text-align:center">Part 3: Saving Co-ordinates</h1>

## 1. Creating GUI for marking locations

* We create a new gui in the multimap_server package using tkinter, yaml and tf2_ros.

![Marking GUI](images/location_marking_gui.png)

* Create `location_marker.py` in `multimap_server/scripts` folder and copy paste the following code ->

```python
#!/usr/bin/env python

import Tkinter as tk
import rospy
import os
import tf2_ros
import tf_conversions
import yaml
import sys

class LocationMarker(tk.Frame):

    def __init__(self, master=None):
        tk.Frame.__init__(self, master=master)
        self.tfBuffer = tf2_ros.Buffer()
        self.file_path = None
        self.map = {"locations": {}}
        self.tfListener = tf2_ros.TransformListener(self.tfBuffer)
        self.pack(expand=True, fill="both")
        self.create_widgets()

    def save_yaml(self):
        '''
        Save contents of map to yaml file
        '''
        with open(os.path.abspath(self.file_path), 'w') as f:
            yaml.dump(self.map, f)

    def mark_location(self, location_name):
        '''
        Assign location_name with x, y, theta to map using lookup_transform between map and dbot/base_link.
        Save value to yaml file.        
        '''
        trans = self.tfBuffer.lookup_transform(
            'map',
            'dbot/base_link',
            rospy.Time()
        )
        self.map["locations"][location_name] = {
            "x": trans.transform.translation.x,
            "y": trans.transform.translation.y,
            "a": tf_conversions.transformations.euler_from_quaternion(
                [
                    trans.transform.rotation.x,
                    trans.transform.rotation.y,
                    trans.transform.rotation.z,
                    trans.transform.rotation.w,
                ]
            )[2]
        }
        rospy.loginfo("Location %s marked at x:%f y:%f a:%f" % (
            location_name,
            self.map["locations"][location_name]["x"],
            self.map["locations"][location_name]["y"],
            self.map["locations"][location_name]["a"]
            )
            )

        self.save_yaml()

    def load_map(self, file_path):
        '''
        Set map(dictionary) using values from map file
        '''
        self.file_path = file_path
        with open(os.path.abspath(file_path), 'r') as f:
            self.map = yaml.safe_load(f)

    def create_widgets(self):
        '''
        Code for creating widgets
        '''
        self.var_location_name = tk.StringVar()
        frame = tk.Frame(self)
        frame.pack(side="top", expand=True, fill="both")
        def mark_location_helper():
            if (self.var_location_name.get() != ""):
                self.mark_location(self.var_location_name.get())
                location_entry.delete(0, tk.END)

        location_label = tk.Label(frame, text="Name")
        location_label.grid(column=0, row=0, sticky="ew")

        location_entry = tk.Entry(frame, bd=2, textvariable=self.var_location_name)
        location_entry.grid(column=1, row=0, sticky="ew")

        submit_btn = tk.Button(frame, text="Mark", command=mark_location_helper)
        submit_btn.grid(column=2, row=0, sticky="ew")

if __name__=="__main__":
    rospy.init_node("location_marker", anonymous=True)
    root = tk.Tk()
    root.resizable(False, False)
    app = LocationMarker(master=root)
    app.load_map(sys.argv[1])
    tk.mainloop()
```

## 2. Update CMakeLists.txt
* Add script
```
catkin_install_python(PROGRAMS
    scripts/multimap_server_node.py
    scripts/location_marker.py
    DESTINATION ${CATKIN_PACKAGE_BIN_DESTINATION}
)
```
* `cd` to `multimap_server/scripts` and run `chmod +x location_marker.py` to make the script executable.

## 3. Testing
* First we create a yaml file `locations_test.yaml` with initial contents
```
locations: {}
```
* Start simulation using `roslaunch deliverybot_simulations building_world.launch`
* Run location_marker.py using 
```
rosrun multimap_server location_marker.py "locations_test.yaml"
```
with the path of the yaml file.
* Move robot around and mark important locations
---

<h1 style="text-align:center">Part 4: Using it all together</h1>

1. In a shell, start simulation using
```
roslaunch deliverybot_simulations building_world.launch
```
2. Spawn Package using
```
rosrun deliverybot_simulations package_spawner.py
```
3. Ensure `MAPS_FOLDER_PATH` variable is set correctly by running
`echo $MAPS_FOLDER_PATH`
4. Start map server. Run `multimap_server multimap_server_node.py`
5. Load Map
```
rosservice call /multimap_server/load_map "map_file_path: '$MAPS_FOLDER_PATH/building/lobby.yaml'"
```
6. Open locations.yaml file. Right now we'll use the location file to get the co-ordinates of the locations and send navigation goals manually. You can also use this file in conjunction with other .yaml files to create a complex mapping method.
7. Send navigation goals
    * Using the co-ordinates we can send navigation goals individually [[1]](https://hotblackrobotics.github.io/en/blog/2018/01/29/action-client-py)[[2]](https://github.com/ammaar8/rdd21_deliverybot/blob/main/deliverybot_navigation/scripts/send_goal.py)
    * To send a goal we lookup the co-orinates in the locations.yaml file and run 
    ```python
    rosrun deliverybot_navigation send_goal.py _goal:="{a: 1.583, x: 0.015, y: -2.014}" #example location
    ```
8. In this way we can send goals, control elevator and maps manually to navigate the robot from the drop location to the target apartment.

<h3 style="text-align:center">Services Reference Table</h3>

|Action                  |Service                                                                  |
|:-----------------------|:------------------------------------------------------------------------|
|Clear Costmaps          |`rosservice call /move_base/clear_costmaps`                              |
|Open Elevator Doors     |`rosservice call /elevator/open_eleavtor_doors`                          |   
|Close Elevator Doors    |`rosservice call /elevator/close_elevator_doors`                         |
|Elevator Change Floor   |`rosservice call /elevator/elevator_goto_floor "floor: 0"`               |
|Robot Drop Package      |`rosservice call /dbot/deliver_package`                                  |
|Load Map                |`rosservice call /multimap_server/load_map "map_file_path: '/path/to/file'"`|

<h1 style="text-align:center">References</h1>

[1] [Reference for sending navigattion goals](https://hotblackrobotics.github.io/en/blog/2018/01/29/action-client-py).
[2] [send_goal.py](https://github.com/ammaar8/rdd21_deliverybot/blob/main/deliverybot_navigation/scripts/send_goal.py).
[3] [Github Reop](https://github.com/ammaar8/rdd21_deliverybot).